In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import glob
import os
from PIL import Image

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
torch.manual_seed(0)

def show_tensor_images(x_real, x_fake):
    image_tensor = torch.cat(x_real[:1, ...], x_fake[:1, ...], dim=0)
    image_tensor = (image_tensor + 1) / 2
    image_tensor = image_tensor.detach().cpu()
    image_grid = make_grid(image_tensor, nrow=1)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

class ImageDataset(Dataset):
    def __init__(self, root, transform=None, mode="train"):
        super().__init__()
        self.transform = transform
        self.filesA = sorted(glob.glob(os.path.join(root, f"{mode}A") + '/*.*'))
        self.filesB = sorted(glob.glob(os.path.join(root, f"{mode}B") + '/*.*'))
        if len(self.filesA) > len(self.filesB):
            self.filesA, self.filesB = self.filesB, self.filesA
        self.new_perm()
        assert len(self.files_A) > 0, "Make sure you downloaded the images!"

    def new_perm(self):
        self.randperm = torch.randperm(len(self.filesB))[:len(self.filesA)]
                                                         
    def __getitem__(self, index):
        itemA = self.transform(Image.open(self.filesA[index % len(self.filesA)]))
        itemB = self.transform(Image.open(self.filesB[self.randperm[index]]))
        if itemA.shape[0] != 3:
            itemA = itemA.repeat(3,1,1)
        if itemB.shape[0] != 3:
            itemB = itemB.repeat(3,1,1)
        if index == len(self) - 1:
            self.new_perm()
        return itemA, itemB

    def __len__(self):
        return min(len(self.filesA), len(self.filesB))



In [ ]:
if len(os.listdir(".")) < 3:
    !wget https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
    !unzip horse2zebra.zip

In [ ]:
class AdpativeInstanceNorm2d(nn.Module):
    